In [15]:
# 1. import할 라이브러리와 변수
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
RESULT_PATH = '/content/drive/My Drive/Colab Notebooks/Final_project/data/output'
now = datetime.now() #파일이름 현 시간으로 저장하기

'''
requests, bs4  >> 크롤링을 위한 라이브러리

pandas >> csv파일을 불러오기 위한 라이브러리

datetime >> 최종 저장파일 이름을 현시각으로 저장하기 위한 라이브러리

RESULT_PATH >> 크롤링 결과를 저장할 폴더 경로(꼭 확인해서 변경해줄것)!!

'''

'\nrequests, bs4  >> 크롤링을 위한 라이브러리\n\npandas >> csv파일을 불러오기 위한 라이브러리\n\ndatetime >> 최종 저장파일 이름을 현시각으로 저장하기 위한 라이브러리\n\nRESULT_PATH >> 크롤링 결과를 저장할 폴더 경로(꼭 확인해서 변경해줄것)!!\n\n'

In [20]:
# 2. main 함수 
'''
사용자로부터 입력 값을 가져와 crawler함수로 넘겨주고 
엑셀로 만들라는 지시를 한다
'''

def main():
    maxpage = input("최대 출력할 페이지수 입력하시오: ")
    query = input("검색어 입력: ")
    s_date = input("시작날짜 입력(2019.01.01):") #2020.01.01
    e_date = input("끝날짜 입력(2019.04.28):") #2020.04.04
    crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다.
    excel_make() #엑셀로 만들기
main()


최대 출력할 페이지수 입력하시오: 1000
검색어 입력: 이낙연
시작날짜 입력(2019.01.01):2020.01.01
끝날짜 입력(2019.04.28):2020.04.04
1
https://search.naver.com/search.naver?where=news&query=이낙연&sort=0&ds=2020.01.01&de=2020.04.04&nso=so%3Ar%2Cp%3Afrom20200101to20200404%2Ca%3A&start=1
11
https://search.naver.com/search.naver?where=news&query=이낙연&sort=0&ds=2020.01.01&de=2020.04.04&nso=so%3Ar%2Cp%3Afrom20200101to20200404%2Ca%3A&start=11
21
https://search.naver.com/search.naver?where=news&query=이낙연&sort=0&ds=2020.01.01&de=2020.04.04&nso=so%3Ar%2Cp%3Afrom20200101to20200404%2Ca%3A&start=21
31
https://search.naver.com/search.naver?where=news&query=이낙연&sort=0&ds=2020.01.01&de=2020.04.04&nso=so%3Ar%2Cp%3Afrom20200101to20200404%2Ca%3A&start=31
41
https://search.naver.com/search.naver?where=news&query=이낙연&sort=0&ds=2020.01.01&de=2020.04.04&nso=so%3Ar%2Cp%3Afrom20200101to20200404%2Ca%3A&start=41
51
https://search.naver.com/search.naver?where=news&query=이낙연&sort=0&ds=2020.01.01&de=2020.04.04&nso=so%3Ar%2Cp%3Afrom20200101to20200404%2Ca

FileNotFoundError: ignored

In [0]:
# 3. crawler 함수
'''
네이버 뉴스 홈에 등록된 기사인지 확인하고
맞으면 해당 url을 get_news에 보내고 리턴값(리스트 형식)을 메모장으로 저장하는 함수
'''
def crawler(maxpage,query,s_date,e_date):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1 # 11= 2페이지 21=3페이지 31=4페이지 ...81=9페이지 , 91=10페이지, 101=11페이지
    f = open("/content/drive/My Drive/Colab Notebooks/Final_project/data/output/contents_text.txt", 'w', encoding='utf-8')

    while page < maxpage_t:
        print(page)
        
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)

        req = requests.get(url)
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
            #print(soup)

        for urls in soup.select("._sp_each_url"):
            try :
                #print(urls["href"])
                if urls["href"].startswith("https://news.naver.com"):
                #print(urls["href"])
                    news_detail = get_news(urls["href"])
                    # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3])) # new style
            except Exception as e:
                print(e)
                continue
        page += 10

    f.close()


In [0]:
# 4. get_news함수 
'''
네이버 뉴스 홈에 등록된 기사를 크롤링해오는 함수

- crawler 함수로 부터 n_url을 받아서 크롤링을 해온다.

- 결과를 new_detail이라는 list로 저장하여 리턴 한다.

'''
def get_news(n_url):
    news_detail = []
    
    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')
    
    title = bsoup.select('h3#articleTitle')[0].text #대괄호는 h3#articleTitle 인 것중 첫번째 그룹만 가져오겠다.
    news_detail.append(title)
    
    pdate = bsoup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate)
    
    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
    
    news_detail.append(n_url)
    
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)
    
    return news_detail



In [0]:
# 5. excel_make 함수
'''
  위에서 만든 메모장 파일을 csv 형식으로 불러와서 data 변수에 저장한다.

data 변수에 columns명을 주고 excel 파일로 저장한다.

이때 이름은 datetime 라이브러리의 now함수를 사용하여 현시각으로 저장한다.
'''

def excel_make():
    data = pd.read_csv(RESULT_PATH+'contents_text.txt', sep='\t',header=None, error_bad_lines=False)
    data.columns = ['years','company','title','contents','link']
    print(data)

    xlsx_outputFileName = '%s-%s-%s %s시 %s분 %s초 result.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    #xlsx_name = 'result' + '.xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8')